# SageMaker Bring Your Own Algorithm Container

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Test-on-Local-Machine" data-toc-modified-id="Test-on-Local-Machine-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Test on Local Machine</a></span><ul class="toc-item"><li><span><a href="#Build-Docker-Image" data-toc-modified-id="Build-Docker-Image-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Build Docker Image</a></span></li><li><span><a href="#Local-Test" data-toc-modified-id="Local-Test-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Local Test</a></span><ul class="toc-item"><li><span><a href="#train_local.sh" data-toc-modified-id="train_local.sh-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span><code>train_local.sh</code></a></span></li><li><span><a href="#serve_local.sh" data-toc-modified-id="serve_local.sh-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span><code>serve_local.sh</code></a></span></li><li><span><a href="#predict.sh" data-toc-modified-id="predict.sh-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span><code>predict.sh</code></a></span></li></ul></li><li><span><a href="#Publish-Image-to-ECR" data-toc-modified-id="Publish-Image-to-ECR-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Publish Image to ECR</a></span><ul class="toc-item"><li><span><a href="#Manual-Steps" data-toc-modified-id="Manual-Steps-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Manual Steps</a></span></li></ul></li></ul></li><li><span><a href="#Train-Model-in-SageMaker" data-toc-modified-id="Train-Model-in-SageMaker-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Train Model in SageMaker</a></span><ul class="toc-item"><li><span><a href="#Set-up-the-environment" data-toc-modified-id="Set-up-the-environment-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Set up the environment</a></span></li><li><span><a href="#Train-Model" data-toc-modified-id="Train-Model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Train Model</a></span></li><li><span><a href="#Create-an-estimator-and-fit-the-model" data-toc-modified-id="Create-an-estimator-and-fit-the-model-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create an estimator and fit the model</a></span></li></ul></li><li><span><a href="#Host-Model-in-SageMaker" data-toc-modified-id="Host-Model-in-SageMaker-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Host Model in SageMaker</a></span><ul class="toc-item"><li><span><a href="#Choose-some-data-and-use-it-for-a-prediction" data-toc-modified-id="Choose-some-data-and-use-it-for-a-prediction-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Choose some data and use it for a prediction</a></span></li><li><span><a href="#Optional-cleanup" data-toc-modified-id="Optional-cleanup-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Optional cleanup</a></span></li></ul></li><li><span><a href="#Run-Batch-Transform-Job" data-toc-modified-id="Run-Batch-Transform-Job-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run Batch Transform Job</a></span><ul class="toc-item"><li><span><a href="#Create-a-Transform-Job" data-toc-modified-id="Create-a-Transform-Job-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Create a Transform Job</a></span></li><li><span><a href="#View-Output" data-toc-modified-id="View-Output-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>View Output</a></span></li></ul></li></ul></div>

## Test on Local Machine

### Build Docker Image

Build the image using Dockerfile in `container` folder.

```sh
cd container
docker build -t sagemaker_bring_your_own . 
```

### Local Test

To test the algorithm and docker image, use the three shell scripts in the **`test`** folder. It builds the image and runs it in a container to train and test the model. It mounts a directory structure that mimics production.

#### `train_local.sh`

- Run the script with the name of the image. 
- It maps `test_dir` folder to `/opt/ml` folder. 
- Test data is placed in `test_dir/input/data`.
- (Optional) Modify the file `test_dir/input/config/hyperparameters.json` to have the hyperparameter settings that you want to test (as strings).
- Trained model will be saved to `test_dir/models` folder.

```sh
./train_local.sh sagemaker_bring_your_own
```

#### `serve_local.sh`

- Run this with the name of the image to serve the model after model is trained.

```sh
./serve_local.sh sagemaker_bring_your_own
```

#### `predict.sh`

- Run this with the name of a payload file and (optionally) the HTTP content type you want. The content type will default to `text/csv`. For example, you can run 

```sh
./predict.sh payload.csv text/csv
```

- Alternatively, can run following command to test the prediction. Need to use full path in the curl command.

```sh
curl --data-binary @D:/tmp/sagemaker_bring_your_own/container/local_test/payload.csv -H "Content-Type: text/csv" -v http://localhost:8080/invocations
```

### Publish Image to ECR

Run the `build_and_push.sh <IMAGE_NAME>` script in the folder `container`.

```
./build_and_push.sh sagemaker_bring_your_own
```


#### Manual Steps

For debugging purpose, you can also run following commands one by one.

1. With AWS CLI 2, login into AWS ECR.

```sh
aws ecr get-login-password --region ap-southeast-1 | docker login --username AWS --password-stdin <ACCOUNT_ID>.dkr.ecr.ap-southeast-1.amazonaws.com
```

2. Tag local image with full ECR image name.

```sh
docker tag sagemaker_bring_your_own <ACCOUNT_ID>.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker_bring_your_own:latest
```

3. Push image to ECR.

```sh
docker push <ACCOUNT_ID>.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker_bring_your_own:latest
```

## Train Model in SageMaker

After local test, use SageMaker to train models and use the model for hosting or batch transforms.

### Setup the environment

#### Copy Files to S3

- Copy `test/test_dir/input` folder into the working S3 bucket, e.g. `s3://temp-305326993135/sagemaker_bring_your_own/`
- Copy `test/data` folder into the working s3 bucket, e.g. ``s3://temp-305326993135/sagemaker_bring_your_own/`

#### Setup SageMaker Notebook Instance

- Create a SageMaker Notebook instance
- Upload this Jupyter Notebook file

### Initialize Variables

- Import libraries
- Get SageMaker execution role
- Get current AWS region

In [46]:
import boto3
import re
import os
import numpy as np
import pandas as pd
import sagemaker
import json
from time import gmtime, strftime

role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name

- Setup S3 data paths to input training data and output model artifacts

In [35]:
s3_client = boto3.client('s3')

IMAGE_NAME = 'sagemaker_bring_your_own'

# Where the training data is located
input_bucket = 'temp-305326993135'
input_data_prefix = f'{IMAGE_NAME}/input/data'
input_config_prefix = f'{IMAGE_NAME}/input/config'

# Where to save code and model artifacts
# output_bucket = sagemaker.Session().default_bucket()
# TEST
output_bucket = 'temp-305326993135'
output_prefix = f'{IMAGE_NAME}'

Read hyper-parameters for the model.

In [52]:
hyperparameters = {}
try:
    hyper_param_file = f'{input_config_prefix}/hyperparameters.json'
    response = s3_client.get_object(Bucket=input_bucket, Key=hyper_param_file)
    content = response['Body']
    hyperparameters = json.loads(content.read())
except Exception as ex:
    pass

### Create an Estimator and Fit the Model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [54]:
full_image_name = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{IMAGE_NAME}:latest"

model = sagemaker.estimator.Estimator(
    base_job_name=IMAGE_NAME.replace('_', '-'),
    image_uri=full_image_name,
    role=role,
    instance_count=1,
    instance_type="ml.c4.2xlarge",
    volume_size= 5,     # GB
    output_path=f"s3://{output_bucket}/{output_prefix}/output",
    sagemaker_session=sagemaker.Session()
)

# Set hyperparameters for the model training
model.set_hyperparameters(**hyperparameters)

# Specify s3 folder which contains training data and its data type
train_input = sagemaker.TrainingInput(s3_data=f's3://{input_bucket}/{input_data_prefix}/train/', 
                                      content_type='text/csv')

model.fit({'train': train_input}, wait=True)


2022-01-18 07:53:46 Starting - Starting the training job...
2022-01-18 07:54:09 Starting - Launching requested ML instancesProfilerReport-1642492425: InProgress
...
2022-01-18 07:54:44 Starting - Preparing the instances for training.........
2022-01-18 07:56:10 Downloading - Downloading input data
2022-01-18 07:56:10 Training - Downloading the training image..Rows in training data 150
Starting the training.
Training complete.

2022-01-18 07:56:46 Uploading - Uploading generated training model
2022-01-18 07:57:10 Completed - Training job completed
Training seconds: 61
Billable seconds: 61


## Host Model in SageMaker
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [81]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer

predictor = model.deploy(initial_instance_count=1, 
                         instance_type="ml.m4.xlarge", 
                         serializer=CSVSerializer())

----!

In [82]:
endpoint_name = predictor.endpoint_name
print(f'Endpoint: {predictor.endpoint_name}')

Endpoint: sagemaker-bring-your-own-2022-01-18-08-36-25-531


### Choose some data and use it for a prediction

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [83]:
test_data = pd.read_csv("payload.csv", header=None)
test_data.sample(3)

,0,1,2,3
0,5.0,3.5,1.3,0.3
23,6.8,3.2,5.9,2.3
21,6.9,3.1,5.1,2.3


Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

In [89]:
result = predictor.predict(data=test_data.values).decode('utf-8')
result = result.split()
print(result)

['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica']


### Optional Cleanup
When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

## Run Batch Transform Job
You can use a trained model to get inference on large data sets by using [Amazon SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html). A batch transform job takes your input data S3 location and outputs the predictions to the specified S3 output folder. Similar to hosting, you can extract inferences for training data to test batch transform.

### Create a Transform Job
We'll create an `Transformer` that defines how to use the container to get inference results on a data set. This includes the configuration we need to invoke SageMaker batch transform:

* The __instance count__ which is the number of machines to use to extract inferences
* The __instance type__ which is the type of machine to use to extract inferences
* The __output path__ determines where the inference results will be written

In [94]:
transform_bucket = 'temp-305326993135'
transform_prefix = f'{IMAGE_NAME}/data'
transform_output_path = f's3://{transform_bucket}/{transform_prefix}'
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    assemble_with="Line",
    accept="text/csv",
    output_path=transform_output_path,
)

We use tranform() on the transfomer to get inference results against the data that we uploaded. You can use these options when invoking the transformer. 

* The __data_location__ which is the location of input data
* The __content_type__ which is the content type set when making HTTP request to container to get prediction
* The __split_type__ which is the delimiter used for splitting input data 
* The __input_filter__ which indicates the first column (ID) of the input will be dropped before making HTTP request to container

In [95]:
transform_input = f's3://{transform_bucket}/{transform_prefix}/iris.csv'
transformer.transform(
    data=transform_input, content_type="text/csv", split_type="Line", input_filter="$[1:]"
)
transformer.wait()

...........................
Starting the inference server with 4 workers.
[2022-01-18 08:57:32 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2022-01-18 08:57:32 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2022-01-18 08:57:32 +0000] [10] [INFO] Using worker: sync
[2022-01-18 08:57:32 +0000] [14] [INFO] Booting worker with pid: 14
[2022-01-18 08:57:32 +0000] [15] [INFO] Booting worker with pid: 15
[2022-01-18 08:57:32 +0000] [17] [INFO] Booting worker with pid: 17
[2022-01-18 08:57:32 +0000] [18] [INFO] Booting worker with pid: 18
Load model from /opt/ml/model/model.pkl
169.254.255.130 - - [18/Jan/2022:08:57:38 +0000] "GET /ping HTTP/1.1" 200 2 "-" "Go-http-client/1.1"
169.254.255.130 - - [18/Jan/2022:08:57:38 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Invoked with 150 records
Perform prediction on 150 rows of input
169.254.255.130 - - [18/Jan/2022:08:57:38 +0000] "POST /invocations HTTP/1.1" 200 1400 "-" "Go-http-client/1.1"
2022-01-18T0

For more information on the configuration options, see [CreateTransformJob API](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTransformJob.html)

### View Output

The output file will be the input file name + `.out`.

In [115]:
s3_client = sagemaker.Session().boto_session.client("s3")
output_file =  f"{transform_prefix}/iris.csv.out"

result = []
response = s3_client.list_objects(Bucket=transform_bucket, Prefix=output_file)
for o in response.get('Contents'):
    data = s3_client.get_object(Bucket=transform_bucket, Key=o.get('Key'))
    contents = data['Body'].read()
    result.extend([line.strip() for line in contents.decode("utf-8").split()])

print(result)

['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'virginica', 'versicolor', 'virginica', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'virginica', 'versicolor', 'versicolor'